In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import ExampleGraphs
import importlib
import Drawing


In [ ]:
#importlib.reload(ExampleGraphs)
#graph_params = ExampleGraphs.TakagakiMaze()

#importlib.reload(Drawing)
graph_params = Drawing.DrawGraph().userDrawGraph()

D = graph_params['D']
L = graph_params['L']
pos = graph_params['pos']

graph = nx.from_numpy_matrix(D)

N = len(L)
src = 0
sink = 1

for round in range(50):
    if(round % 1 == 0):
        # Update nx graph edges for visualization
        for i in range(N):
            for j in range(i, N):
                if(D[i][j] != 0):
                    graph[i][j]['weight'] = D[i][j]
        weights = [graph[u][v]['weight'] for u,v in graph.edges()]

        plt.cla()
        #fig = plt.figure()
        nx.draw(graph, pos, 
                width=list(weights * 100),
                with_labels=True,
                node_color='lightgreen')
        
        plt.savefig(f'images/graph{str(round).zfill(2)}.png', dpi=300)
        
        #fig.show()
        
        #plt.close(fig)
    
    # sum of currents leaving each node using KCL
    sums = np.zeros(N)
    sums[src] = 1
    sums[sink] = -1

    # Conductance of edges = D / L
    cond = np.divide(D,L)

    # Solve for node pressures 

    A = - np.copy(cond)

    for i in range(N):
        A[i][i] = np.sum(cond[i]) - cond[i][i]

    # Pressure of sink node is 0
    A[:, N-1] = 0

    A_inv = np.linalg.pinv(A)

    press = np.matmul(A_inv,sums)

    # Update diameters using slime mold algorithm

    # difference in pressure between nodes
    dP = np.zeros((N,N))
    for i in range(N):
        for j in range(i, N):
            dP[i][j] = press[i] - press[j]
            dP[j][i] = - dP[i][j]

    #print("dP:" + str(dP))

    # Flow through each edge
    Q = cond * dP

    # Update diameters
    D = 0.5*( (Q*dP / (L*press[0])) + D)

In [ ]:
# Create mp4 from images

import subprocess

cmd = "ffmpeg -framerate 2 -i images/graph%2d.png animation.mp4"
returned_value = subprocess.call(cmd, shell=True)  # returns the exit code in unix
print('returned value:', returned_value)